<a href="#Overview"></a>
# Overview
* <a href="#097d73d0-4d4c-4aa6-a25d-1adcb61b80aa">Current Source Density Analysis</a>
  * <a href="#26d4fca1-334c-43e2-9515-ebda1ef79c0c">How can we highlight local aspects of electrophysiology data in the presence of volume conduction? Can we identify the laminar structure of the auditory cortex given a laminar array of electrodes and auditory stimuli?</a>
    * <a href="#ff0e97a8-0717-4ad7-aa9c-0806d2ac44a2">Function list</a>
* <a href="#290fea56-2928-4374-ba9a-3ce61ed5cb1a">Load the data</a>
  * <a href="#1aee6002-90b9-4130-b348-cd114872f81e">Use pycurl to pull down the data</a>
  * <a href="#31d5ecb6-b168-440d-b17d-5d76fc1f054f">Load the data</a>
    * <a href="#4b21d5a4-2041-446f-b74d-e42f907d003c">Exercise 1: load the stimulus epoch data</a>
* <a href="#5c1c7d48-dc77-45fe-838f-d46867b40f31">Filter the signals</a>
  * <a href="#828d393f-1a44-4110-a1ef-964b9568839b">Exercise 2: Create a filter to isolate the high frequency component</a>
  * <a href="#772f4201-1900-4f91-b665-06340a6ed99b">Filter raw data to extract the local field potential and the multiunit activty</a>
    * <a href="#4076b2aa-1859-41d1-92d6-33ef80608f15">Exercise 3: filter for local field potential</a>
    * <a href="#01e76a8f-8a0e-4700-98f9-ea5d2ad31e37">Exercise 4: rectify the MUA signal</a>
* <a href="#40788a7c-dc9a-4e43-8fe8-f2da5f2c3a95">Trial averaging the data</a>
  * <a href="#c90c390a-f134-401c-9486-f71c7ccbcb32">Extracting event data</a>
    * <a href="#60cba457-88ae-4fe5-9cdb-9c14127c05ab">Exercise 5: convert units</a>
    * <a href="#404cfe07-d2dd-4c77-8cba-e642f8442c5b">Extract LFP trials</a>
    * <a href="#fd187b1b-bc68-4350-9136-d5a35b3761aa">Exercise 6: extract MUA trials</a>
    * <a href="#82ad499a-0b48-4a00-b650-d447f4c1d423">Exercise 7: trial average the data</a>
* <a href="#0d000429-4938-4b44-a6cb-ee2cf27ee210">Extracting a single column of electrodes</a>
  * <a href="#6e0f204b-214b-4cc0-a791-8a09aaad16c0">Excercise - extracting a single column of electrodes from the probe.</a>
* <a href="#fe6153e4-21b1-4d2f-ada1-b5fe54d8aa5e">Spatial filtering</a>
* <a href="#1b0038c8-fdb9-4f1d-bdc5-b8f8026fa6a4">Current source density algorithm - estimating the second spatial derivative</a>
  * <a href="#c156ac6f-530e-4809-b285-cf7a7f552ce1">Excercise - initialize an output matrix for CSD</a>
    * <a href="#e616119c-2619-402e-a937-59dc601180b0">Excercise - Using a for loop, compute CSD at all electrode positions</a>
* <a href="#c5f61598-3e3b-4c29-b775-f38abaf3a00e">Plot the estimated CSD</a>
  * <a href="#4baf145a-396d-4e8c-8d45-7aeeaedfc5f7">Exercise 8: zooming in on stimulus window</a>
    * <a href="#3007da04-e724-43bf-a35d-398e9be15d52">Excercise - adding in MUA</a>

<a id="097d73d0-4d4c-4aa6-a25d-1adcb61b80aa"></a>
# Current Source Density Analysis
<a href="#Overview">Return to overview</a>

<a id="26d4fca1-334c-43e2-9515-ebda1ef79c0c"></a>
## How can we highlight local aspects of electrophysiology data in the presence of volume conduction? Can we identify the laminar structure of the auditory cortex given a laminar array of electrodes and auditory stimuli?
<a href="#Overview">Return to overview</a>

Individual neurons act as sources and sinks of current forming dipole moments. Depending on the anatomy of the given brain region being recorded from, neurons are often aligned forming larger cumulative dipoles that are picked up by the recording electrode. If the dipole moment for two neurons are not aligned this can effectively cancel out. Fortunately, cortex has fairly regular laminar structure with many neurons having similar alignments. When a neuron receives input at its dendrites and channels open allowing positively chareged ions in this leads to a negative deflection in the extracellularly recorded potentials (called a current sink), and the extracellular space farther away from this sink will be more positive and will be a source for return current (current source). <i>


<p style="text-align: center;"><img src="images/sources and sinks.png" alt="drawing" width="500"/></p>


The activity at recording electrodes reflects a mixture of local activity and activity from neighboring sources that can propogate via the conductive media of the brain. We want to reduce this volume conducted signal to identify local positive (sources) and negative (sinks) in the LFP signal which can reflect localized neural activity. To address this issue, we can simply take the spatial derivative, the change in voltage per distance, given the known spacing of contact sites. This is typically called current source density analysis when applied to multicontanct probes in the brain and has been used to identify local activity patterns associated with laminar processing. For a more thorough background, see Neuromethods (2012) 67: 205–218 DOI 10.1007/7657_2011_6. 
    
<p style="text-align: center;"><img src="images/laminar_probe_CSD.png" alt="drawing" align="center" width="500"/></p>

We will now go through some of the basic signal processing steps needed to estimate the CSD for laminar probes.

<a id="ff0e97a8-0717-4ad7-aa9c-0806d2ac44a2"></a>
### Function list
<a href="#Overview">Return to overview</a>

Numpy
* `np.sin()`
*  `np.pi()`
*  `np.hanning()`
*  `np.matlib.repmat()`

Matplotlib (these are not central to today's presentation but are good to know about). We'll mostly be using plot functions that we already know.
* `plt.colorbar()`
*  `plt.set_labels()`

Scipy
* `scipy.signal.butter()`
*  `scipy.signal.sosfilt()`
*  `scipy.signal.convolve2d()`

pycurl
* `pycurl.Curl`
* `setopt`
* `perform`
* `close`

<a id="290fea56-2928-4374-ba9a-3ce61ed5cb1a"></a>
# Load the data
<a href="#Overview">Return to overview</a>


First, import the requisite libraries. You should already have all these installed in your Anaconda environment.  Including `pycurl`, which is new for today's class.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import scipy as scipy
import tables as tb
from scipy.signal import convolve2d, butter, sosfilt
import pycurl

<a id="1aee6002-90b9-4130-b348-cd114872f81e"></a>
## Use pycurl to pull down the data
<a href="#Overview">Return to overview</a>

The data file for this exercise is too large to include in a github package, so we need to download it from a different server. This situation is not that uncommon, as often you find yourself running analysis on a different computer from where the data are stored. The `pycurl` library provides a simple way of using the `http` protocol to download from a web server.

Also note:  Today's datafile is nearly 200MB in size. If you don't have that much free space on your computer, you might need to work with someone else.

In [ ]:
with open('data/TAR010a.h5', 'wb') as f:
    c = pycurl.Curl()
    c.setopt(c.URL, 'https://hearingbrain.org/tmp/TAR010a.h5')
    c.setopt(c.WRITEDATA, f)
    c.perform()
    c.close()

<a id="31d5ecb6-b168-440d-b17d-5d76fc1f054f"></a>
## Load the data
<a href="#Overview">Return to overview</a>

Raw (1500Hz lowpass filtered and downsampled for sake of convenience)- data from a 64 channel silicone probe recording made in ferret primary auditory cortex has been stored in a .hdf5 file in the `data/` subdirectory thanks to the pycurl code above. This is the same general file type as the data from week 4 image processing and can be accessed with the tables module, tb. This file should contain the raw voltage data and epoch timestamps surrounding the stimuli.

Remember, the tb module has a function, `open_file`, that will let us store a "File" object needed to access the data. Lets load this file and print out the directories so we know where to pull in data from.

In [ ]:
#load file
fh = tb.open_file('data/TAR010a.h5')

#print file directories
print(fh)

Look at the output of the file above. The `tb` file object also has a method, `get_node` that returns a node object. This node object provides a method, `read`, that loads the data from disk and returns it as a Numpy array object.

Use the 'get_node' and 'read' functions to load the data into a numpy array object called "probedata'.

In [ ]:
node = fh.get_node('/probedata')
probedata = node.read()

Check to make sure you've got the right shape

In [ ]:
print(probedata.shape)

<a id="4b21d5a4-2041-446f-b74d-e42f907d003c"></a>
### Exercise 1: load the stimulus epoch data
<a href="#Overview">Return to overview</a>

Repeat the above steps to load the data for "epochs" from the file object into a variable called `epochs`. Check its shape. This matrix tells us the start and stop time of each stimulus event.

In [ ]:
# Your answer here

When you're done, close the file for good measure.

In [ ]:
fh.close()

<a id="5c1c7d48-dc77-45fe-838f-d46867b40f31"></a>
# Filter the signals
<a href="#Overview">Return to overview</a>


Often times, we have data that we know has multiple unique sources in it, noise, or components that are not of interest given a particular question. Different sources of noise occur on different timescales. In extracellular electrophysiology data there are often multiple components we are interested in, AP/spikes - voltage fluctaions on the order of 1-2ms due to fast Na+/K+ channel dynamics and the local field potential (LFP) - thought to primarily reflect slower timescale channels (AMPA/GABA) as well as synchronous spiking near the electrode (10-100 ms). Although somewhat arbitrary, we can typically separate these different features of interest by filtering the data. Here we will briefly go over signal filtering which is an important step in many signal processing pipelines.

To demonstrate the basics of signal filtering, lets create a simple signal composed of two components with different frequency characterstics and then use a filter to separate them. Notice how the high-freuqency component "rides" on top of the low frequency component when they're added together.

In [ ]:
#simulation parameters
freq1 = 10    # 10 cycles/sec or 10 Hz -- slow signal
freq2 = 800   # 800 Hz, fast signal
fs = 1000     # sampling rate
time = np.arange(0, 1, 1/fs)  # x values over a 1-second time period

#create signals
signal1 = np.sin(2*np.pi*freq1*time)
signal2 = np.sin(2*np.pi*freq2*time)

#combine signals
combined = signal1 + signal2

#plot combined signal
f,ax = plt.subplots(3,1,figsize=(15,6))
ax[0].plot(time, signal1, color='r')
ax[0].set_ylabel('10 Hz component')
ax[1].plot(time, signal2, color='b')
ax[1].set_ylabel('800 Hz component')
ax[2].plot(time, combined, color='k')
ax[2].set_ylabel('combined signal')
ax[2].set_xlabel('time (s)');

Notice that a 10 Hz signal oscillates 10 times in this one-second period. The 800 Hz component oscillates 800 times, if you want to count.

Now lets suppose we only have the final combined singal, and use a filter to separate the two known sources.

A **butterworth filter** is a commonly used filter that has a flat profile in the pass band, the frequencies that it lets
through won't have amplitude distortions or ringing effects that some other filter kernels create. Scipy has a nice function, `scipy.signal.butter()`, that lets us design our own butterworth filter.

There are two major parameters to be considered for a filter. First is the filter order, which is essentially the length of our filter kernel. This parameter specifies length based on a multiple of the the lowest frequency asked for in the cutoff frequency. In general, this should be larger than 3, which is the default. The larger the filter order, the better the filter frequency precision although this comes at a drawback of increased computation time and creates a shift in the phase of the signal (this can be corrected for so it is not a big deal). The other parameters are the cutoff frequencies/passband which specifies the frequencies that we want to filter out of our signal. The next parameter is the filter type (low, high, bandpass). You also need to know the sampling frequency of the signal you wish to filter. One common way to implement a filter is a second order sections method which we implement here.

For more information regarding filters, I'd recommend Mike X Cohens book analyzing neural time series which has a whole chapter devoted to properly designing filters. Additonally, The scientists and engineers guide to digital signal processing is freely available although a bit more dense and less fun https://www.dspguide.com/.

In [ ]:
#filter parameters
order = 5
passband = 50

#design filter
sos = scipy.signal.butter(order, passband, 'lowpass', fs=fs, output='sos')

#filter data
signal1filt = scipy.signal.sosfilt(sos, combined)

#plot signals
f,ax = plt.subplots(2,1,figsize=(15,4))
ax[0].plot(time, combined, color='k')
ax[0].set_ylabel('combined')
ax[1].plot(time, signal1filt, color='k', label='low-pass filtered')
ax[1].plot(time, signal1, color='r', label='original')
ax[1].legend()
ax[1].set_xlabel('time (s)');

This example is extremely basic, but it shows that our filter can essentialy extract one signal from a combined signal. Which is pretty cool.

You might notice that there is slight temporal shift in the filtered signal. This "phase shift", reflecting the order of the filter, can be important for some analyses, and it's good to be aware that this can happen. However, it's not a huge concern for today's exercises.

<a id="828d393f-1a44-4110-a1ef-964b9568839b"></a>
## Exercise 2: Create a filter to isolate the high frequency component
<a href="#Overview">Return to overview</a>


Now that you have some experience with filtering, design a filter to extract the high frequency component of the combined signal we generated and save it as the vector `signal2filt`.

In [ ]:
# Your answer here

Then plot the first 100 samples of your filtered signal and the original high frequency component.

In [ ]:
#plot signals
f,ax=plt.subplots(2,1,figsize=(15,4))
ax[0].plot(time[:100], combined[:100], color='k')
ax[0].set_ylabel('combined')

ax[1].plot(time[:100], signal2filt[:100], color='k', label='high-pass filtered')
ax[1].plot(time[:100], signal2[:100], color='b', label='original')
ax[1].legend()
ax[1].set_xlabel('time (s)');

<a id="772f4201-1900-4f91-b665-06340a6ed99b"></a>
## Filter raw data to extract the local field potential and the multiunit activty
<a href="#Overview">Return to overview</a>


As stated earlier, today's data is sampled at 1500Hz. This means the signal should contain the low frequency signal neuroscientists typically refer to as the local field potential (LFP) as well as higher frequency spiking related activity although the sampling rate and filter are much too low to identify spike waveforms we will use this activity as a measure of what is typically refered to as general spike related activity or multiunit activity (MUA). 

Given our experience with butterworth filters, lets use a high-pass filter to create a signal called `mua` that contains data high-pass filtered above 100Hz. It is worth noting that our data matrix currently has two dimensions. The `sos` function can be applied along either of them. We want to filter along the time dimension so we will need to use an additional input in sosfilt to specify the `axis` we want to filter along. Remember that the resulting `mua` matrix should have the same size as `probedata`.

In [ ]:
fs = 1500
sos = scipy.signal.butter(4, 100, 'highpass', fs=fs, output='sos')
mua = scipy.signal.sosfilt(sos, probedata, axis=1)

The resulting `mua` has slow oscillations removed. We can see that if we look at 1 second of the first channel.

In [ ]:
time = np.arange(fs)/fs
plt.figure(figsize=(15,2))
plt.plot(time, probedata[0,:fs], color='k', label='raw')
plt.plot(time, mua[0,:fs], color='r', label='mua')
plt.legend()
plt.xlabel('time (s)');
plt.ylabel('amplitude (uV)');

<a id="4076b2aa-1859-41d1-92d6-33ef80608f15"></a>
### Exercise 3: filter for local field potential
<a href="#Overview">Return to overview</a>

Now let's extract the local field potential activity in a variable called "lfp" that is bandpassfiltered between 1 and 150Hz. Again, we want to filter along the time dimension so we will need to use an additional input in sosfilt to specify the axis we want to filter along.

In [ ]:
# Your answer here

Complementary to the MUA, the resulting `lfp` has only slow oscillations.

In [ ]:
time = np.arange(fs)/fs
plt.figure(figsize=(15,2))
plt.plot(time, probedata[0,:fs], color='k', label='raw')
plt.plot(time, lfp[0,:fs], color='b', label='lfp')
plt.legend()
plt.xlabel('time (s)');
plt.ylabel('amplitude (uV)');

<a id="01e76a8f-8a0e-4700-98f9-ea5d2ad31e37"></a>
### Exercise 4: rectify the MUA signal
<a href="#Overview">Return to overview</a>

One more detail of filtering. Spiking activity has negative and positive deflections and we want to convert this into an amplitude profile for cleaner viewing and interpretation. To do this, we will take the absolute value of the mua signal with numpy's abs function. `np.abs(x)` will take the absolute value of `x`. Try this for the filtered mua and save the value back into `mua`.

In [ ]:
# Your answer here

<a id="40788a7c-dc9a-4e43-8fe8-f2da5f2c3a95"></a>
# Trial averaging the data
<a href="#Overview">Return to overview</a>


Although you can compute CSD estimates for individual trials, CSD is often estimated on trial-averaged data. We have a list of start and stop times defining trial epochs around an auditory stimulus. The epochs contain a 0.5 second prestimulus silence followed by a tone and post stimulus silence period. The shape of the `epochs` array is a 60 x 2 array. The first column contains start times, and the second column contains stop times.

Subtract the stop times from the start times to determine how long each epoch is. 

In [ ]:
print((epochs[:, 1] - epochs[:, 0]))

<a id="c90c390a-f134-401c-9486-f71c7ccbcb32"></a>
## Extracting event data
<a href="#Overview">Return to overview</a>


<a id="60cba457-88ae-4fe5-9cdb-9c14127c05ab"></a>
### Exercise 5: convert units
<a href="#Overview">Return to overview</a>

Given the knowledge about the epochs, lets use the first and last value in each row of the epochs data (the start and end of the trial epoch), to slice out these epochs for both mua and lfp. 

First, lets convert epoch times into samples that we can use for slicing. Given the sampling rate of 1500 samples per second convert the `epochs` time list from units of seconds to units of time bins, and save it in a new matrix, `epochs_bins`

In [ ]:
# Your answer here

<a id="404cfe07-d2dd-4c77-8cba-e642f8442c5b"></a>
### Extract LFP trials
<a href="#Overview">Return to overview</a>

Now that we have the epochs in samples this should be a simple job to loop through this list of trials, take the start and stop sample for each trial, and slice them out of our longer data structure to generate a list of trial epoched data. Here is how I chose to do this for the lfp signal.

In [ ]:
#First, initialize a matrix that is the size of the expected output. We know there are 60 trials from the epochs list.
#We also know that there are 64 channels of data. Finally, we know that the length of each epoch is 1.6 seconds at 1500hz.

trial_len = int(1.6*1500)
lfp_trials = np.zeros((60, 64, trial_len))  # 3d array: trial X channel X time 

#loop through list of trial epochs and for each epoch extract the data within the epoch bounds
for i in range(60):
    lfp_trials[i, :] = lfp[:, round(epochs_bins[i, 0]):round(epochs_bins[i, 1])]
    
#sanity check that data shape is correct
np.shape(lfp_trials)

<a id="fd187b1b-bc68-4350-9136-d5a35b3761aa"></a>
### Exercise 6: extract MUA trials
<a href="#Overview">Return to overview</a>

Beautiful. We now have a 60 trial, by 64 channel, by 2400 sample matrix for lfp. Using the same loop formulation, now extract the data for the mua and save it as `mua_trials`.  Confirm that the shape is the same as `lfp_trials`.

In [ ]:
# Your answer here

<a id="82ad499a-0b48-4a00-b650-d447f4c1d423"></a>
### Exercise 7: trial average the data
<a href="#Overview">Return to overview</a>


Recall the np.mean() function. We can use this to calculate the trial averaged activity for both our lfp and mua if we specifiy the proper axis. (Which axis indexes trials?) Try taking the mean of this data along the trials axis and save the data into variables "lfp_tavg" and "mua_tavg".

In [ ]:
# Your answer here

Lets now plot the data for one channel to see what it looks like.

In [ ]:
time = np.arange(lfp_tavg.shape[1])/fs
plt.plot(time,lfp_tavg[0, :])
plt.plot(time,mua_tavg[0, :])

<a id="0d000429-4938-4b44-a6cb-ee2cf27ee210"></a>
# Extracting a single column of electrodes
<a href="#Overview">Return to overview</a>


The method of CSD estimation that we are using is easiest to implement for uniform grids. The probes in this dataset have an unique arrangement with 3 columns of uniformly spaced electrodes that are 50 microns apart.

![title](images/64chprobe.png)

We'll extract a single uniformly spaced column and estimate the CSD from that subset of the data. Channel mapping is complicated and requires careful note-keeping. We've got that worked out and the channnels belonging to each column are encoded here.

In [ ]:
#Here are the channel numbers for each of the the three columns of electrodes
left_ch_nums = np.arange(3,64,3)-1
right_ch_nums = np.arange(4,65,3)-1
center_ch_nums = np.insert(np.arange(5, 63, 3),obj=slice(0,1),values =[1,2],axis=0)-1

<a id="6e0f204b-214b-4cc0-a791-8a09aaad16c0"></a>
## Excercise - extracting a single column of electrodes from the probe.
<a href="#Overview">Return to overview</a>


The channel maps provided above specify the index values for each channel. Try slicing out the center column of electrodes from the lfp_tavg and mua_tavg data and save as "lfp_tavg_cent" and "mua_tavg_cent". Remember that after trial averaging, we are now left with a 64 channel by 2400 sample matrix. If I wanted to slice out just two channel, channel 0 and 5, I could do this.

In [ ]:
two_channels = lfp_tavg[[0, 5], :]

Try slicing out the center column of electrodes from the lfp_tavg and mua_tavg data and save as `lfp_tavg_cent` and `mua_tavg_cent`. How big are our data matrices, after we've extracted just this subset of channels?

In [ ]:
# Your answer here

Here's a quick and dirty plot of what all those channels look like.

In [ ]:
time = np.arange(lfp_tavg.shape[1])/fs
plt.plot(time,np.transpose(lfp_tavg_cent));

<a id="fe6153e4-21b1-4d2f-ada1-b5fe54d8aa5e"></a>
# Spatial filtering
<a href="#Overview">Return to overview</a>


We are also going to apply another filter to smooth the data between electrodes. To do this, we generate a hanning window, a tapered cosine function, and by applying this taper across the channels each point becomes a weighted sum of itself and its neighbors that is most heavily weighted towards the center. This should smooth or decrease channel to channel variability that may show up in CSD which emphasizes local differences.

Using `np.hanning` we can create a hanning window to use to smooth our data. np.hanning takes one input which determines the number of samples used to create the window. Here is a 30 sample point hanning window.

In [ ]:
spatial_filter = np.hanning(30)[:, np.newaxis]

plt.plot(spatial_filter)
np.shape(spatial_filter)

 We will use 5 for our window size for the actual analysis, which will mean we are estimating each electrode given one contact on either side. `np.newaxis` is a trick to turn the filter into a 2-d matrix, with size 1 on the time axis. When we convolved, below, the smoothing will happen along axis 0, which is space in our data.

In [ ]:
spatial_filter = np.hanning(5)[:, np.newaxis]

plt.plot(spatial_filter)

np.sum(spatial_filter)

Note the sum of the signal is more than 1. If we are multipling each point of the hanning window by an electrode to scale it and then taking the sum of these 5 scaled points/electrodes we will be amplifying our signal. We can properly scale this by normalizing our spacial filter to equal 1. 

In [ ]:
spatial_filter = spatial_filter / spatial_filter.sum()

plt.plot(spatial_filter)
np.sum(spatial_filter)

We will convolve our hanning filter along our data in the channel dimension. Convolution is a sliding dot product, which will take the five points in our hanning window (0, 0.25, 0.5, 0.25, 0) and multiply each of these values by an electrode voltage value prior to taking the sum of these weighted voltage values as the output. This window will slide along our channel columns smoothing it in the spatial dimension.

In [ ]:
#convolution will take the sliding dot product of our filter with our data which smooths the data by estimating each point as the weighted sum of its neighbors.
LFP_formatted = convolve2d(lfp_tavg_cent, spatial_filter, mode='same', boundary='symm')

<a id="1b0038c8-fdb9-4f1d-bdc5-b8f8026fa6a4"></a>
# Current source density algorithm - estimating the second spatial derivative
<a href="#Overview">Return to overview</a>

The second spatial derivative is often approximated by a three point formula involving taking the difference in voltage over space between two pairs of points - (voltage(x + some distance) - voltage(x)) - ((voltage(x) - voltage(x - some distance)). This is then normalized by the square of the distance - (some distance)^2. The distance chosen is a parameter that can vary and may impact the spatial resolution of identified sources and sinks. In the example from Higley, Neuromethods (2020), delta v is 200 microns as they choose to skip the nearest contact site in a probe with 100 micron spacing. This value may vary depending on your recording setup and anatomical considerations. Here we will just set the distance to one contact site.

<p style="text-align: center;"><img src="images/CSD equation.png" alt="drawing" width="500"/></p>


From the above equations, it should be apparrent that we will not be able to estimate the CSD at top or bottom electrode because we don't have a point above or below these electrodes respectively to perform the calculation. Although dubious, it is often the case that groups will extend/duplicate the data from the top and bottom electrodes under the assumption that the voltage recorded at nearby electrodes is not drastically different and therefore will not greatly impact the estimation. We will avoid doing this here and will simply lose data on both of these channels.

We can convert the equation from up above into code with the channel spacing, detla V, set to 50 microns. Also instead of skipping the neighboring site in our estimation lets use the nearest neighbor. Normalize by the square of the delta V.

In [ ]:
#delta z
ch_spacing = 50

#defining electrode sites for the 3 point equation

V0 = LFP_formatted[0 + 1, :] # voltage at the site we want to estimate at
Va = LFP_formatted[0, :]     # voltage at the site above
Vb = LFP_formatted[0 + 2, :] # voltage at the site below

#estimate the CSD at one site with the 3 point equation

CSD_est = (Vb + Va - 2*V0)/(ch_spacing)**2

<a id="c156ac6f-530e-4809-b285-cf7a7f552ce1"></a>
## Excercise - initialize an output matrix for CSD
<a href="#Overview">Return to overview</a>


This only takes the voltage value at one site. We want to calculate this value for all the channels except for the top and bottom most electrodes which can't be estimated at because we don't know the voltage value above or below these points. This can be done by simply placing the code above in a for loop. Lets start by creating a matrix of the expected output size called CSD_est that is full of zeros. What is the expected output size if we drop two channels?

Go ahead and try to do this yourself. 

In [ ]:
# Your answer here

<a id="e616119c-2619-402e-a937-59dc601180b0"></a>
### Excercise - Using a for loop, compute CSD at all electrode positions
<a href="#Overview">Return to overview</a>


Use a `for` loop to fill each channel of the `CSD_est` matrix. This should be simply copying the triple-point CSD equation code and pasting it into a for loop with a proper number of iterations specified and changing the indexing to account for our looping variable on the channel axis (which was fixed at 0 in the initial example).

In [ ]:
# Your answer here

Plotting of CSD is often done with the sign of values flipped so that sinks appear as positive and sources appear negative. Lets quickly do this for the sake of being normal.

In [ ]:
#flip sign of CSD for plotting purposes
CSD_est = -CSD_est

<a id="c5f61598-3e3b-4c29-b775-f38abaf3a00e"></a>
# Plot the estimated CSD
<a href="#Overview">Return to overview</a>


Time to use plt.imshow() to plot the CSD_est we just generated! Remember, imshow defaults to the origin being in the upper right which has be corrected with an additional parameter. Also, in this case, the units of the x and y axis are not the same so we have to adjust the aspect ratio to keep the axes fixed as this parameter defaults to 1 as it assumes we want square pixels as in images. Note we also use the cmap parameter to change the color map used for plotting. And we use the `clim` parameter to force the color to white for values of zero. What happens if we don't use `cmap` and `clim`?

In [ ]:
plt.figure()
cmax = np.max(np.abs(CSD_est))
plt.imshow(CSD_est, origin = 'lower', aspect = 'auto', cmap = 'bwr', clim=[-cmax,cmax]);

<a id="4baf145a-396d-4e8c-8d45-7aeeaedfc5f7"></a>
## Exercise 8: zooming in on stimulus window
<a href="#Overview">Return to overview</a>

This looks good. However, we are plotting more data than necessary. The majority of the interesting components happen right around sample 750. This corresponds to the stimulus onset at 0.5 second delay (0.5 * 1500 = 750). Lets look at 0.05 seconds prestimulus and 0.15 seconds post stimulus onset. Slice and plot the data around this timeframe to see if it looks better. Save the smaller matrix as `CSD`.

In [ ]:
# Your answer here

Now, try replotting the CSD_est, but this time also adjust the axis with the `extent` parameter of imshow to properly scale the y axis to reflect distance and x axis to reflect the new time window we are focusing on. Also take the time to use plt.xlabel, and plt.ylabel to properly label the axis.

In [ ]:
extent = [-.05, .15, 0, 1000]  # [xmin, xmax, ymin, ymax]

In [ ]:
# Your answer here

Let's also plot a colorbar so we can determine the magnitude of the CSD values. Use plt.colorbar to produce a color bar and save this object to a variable called cbar. Once set as an object, we can use the .set_label method on cbar to label colorbar. Set this label to "CSD uV/um^2"

In [ ]:
plt.figure()
plt.imshow(CSD, extent=extent, origin = 'lower', aspect = 'auto', cmap = 'bwr', clim=[-cmax,cmax])
cbar = plt.colorbar()
cbar.set_label("CSD uV/um^2")
plt.xlabel("Time (s)")
plt.ylabel("Position from recording tip (microns)");

Great! But what, if anything, do these stimulus evoked sources and sinks tell us about neuroanatomy? Here is a bit of what is known about auditory evoked CSDs.

"Noise-evoked columnar CSD patterns were used to determine the location of the A1 recording channel. Two CSD signatures were
used to identify L4: A brief current sink first occurs approximately 10 ms after the noise onset, which was used to determine the lower
border of L4 (Kaur et al., 2005). A triphasic CSD pattern (sink-source-sink from upper to lower channels) occurs between 20 ms and
50 ms, where the border between the upper sink and the source was used to define the upper boundary of L4. Normally, 2 channels
were assigned to L4. Other layers were defined relative to the location of L4 (L2/3: 3 channels above L4; L5: 3 channels below L4; L6: 3
channels below L5). CSD-derived layer assignments were cross-validated against sound-evoked MUA response patterns, where L4
and L5 units responded with higher firing rates and shorter latency."
(Guo et al 2017)

Given this knowledge, lets place a vertical line on our plot at 10ms which should be near the first sink in the signal and might help visualize it. Recall that we can use axvline.

In [ ]:
ch_map = np.arange(0, 20*50, 50)
plt.figure()
plt.imshow(CSD, extent=extent, origin = 'lower', aspect = 'auto', cmap = 'bwr', clim=[-cmax,cmax])
cbar = plt.colorbar()
cbar.set_label("CSD uV/um^2")
plt.xlabel("Time (s)")
plt.ylabel("Position from recording tip (microns)")

# Your answer here

<a id="3007da04-e724-43bf-a35d-398e9be15d52"></a>
### Excercise - adding in MUA
<a href="#Overview">Return to overview</a>


Almost done! Finally, as stated above in the pre-existing knowledge about auditory CSDs, we might expect overlaying the multiunit activity to provide a bit more insight. Lets do that since we took the time to filter our and process the mua. 

To start, we need to separate the mua values from one another - they are all around the same voltage so if we just plotted them they'd all be on top of each other for each channel. Perhaps the simplest thing to do, since the background mua is relatively close to 0, is to add the channel distance to all the values from that channel so they are all evenly spaced apart. 

Lets start by creating a list of channel spacings for all 20 channels in increments of 50. Try to use np.arange() to do this.

In [ ]:
# Your answer here

Next, lets try to add the corresponding distance value from the list of spacings we just generated to each sample value for that channel. This will offset all of our channels from one another by 50 microns. We can use np.repeat to repeat the spacing value from our channel distance list to create an array of equal size to the length of our CSD estimate values for that channel. Below is an example one electrode.

In [ ]:
#First, lets slice it around the stimulus so it matches our CSD plot and drop the top and bottom channel.
mua_spacing = mua_tavg_cent[1:21, prestim:poststim]

Now, lets convert our ch_distance array into a column array. This can be done by adding np.newaxis as a dimension when we save it.

In [ ]:
ch_distance = ch_distance[:, np.newaxis]

This channel spacing is now in the same orientation as our channels in our mua data. If we replicate this matrix until it is the length of our time dimension, we could add all these channel spacings to the mua. np.matlib.repmat(), is a function that as the name implies allows us to replicate a matrix as many times as we want. Try to replicate the ch_distance matrix so it is the same dimensions as our mua_spacing matrix. Then add it to the mua_spacing matrix to space all the channels evenly.

In [ ]:
# Your answer here

Now, lets finally add this mua data to our previous plot. You should be able to copy and paste the plotting from the earlier plot and add an additional plt function within a for loop to plot each channel of spaced mua. 

In [ ]:
time = np.arange(-0.05, 0.15, 1/fs)
plt.figure(figsize=(5,8))
plt.imshow(CSD, extent=extent, origin = 'lower', aspect = 'auto', cmap = 'bwr', clim=[-cmax,cmax])
cbar = plt.colorbar()
cbar.set_label("CSD uV/um^2")

# Your answer here